# Laboratory 1 - Part I (Python)

#### Exercise 1 (adapted from Computer Sciences exam 03/09/2012)

Write a program able to handle the scores of an artistic gymnastic event. The event scores are contained
in a ﬁle whose name is passed from the command line. Every line of the ﬁle contains: the competitor’s
name and surname, competitor’s nationality, and the assigned evaluations provided by 5 judges.

In [75]:
from typing import List, Tuple
from itertools import groupby

filename = "score.txt"

class Athlete:
    def __init__(self, name, surname, nationality, scores):
        self.name = name
        self.surname = surname
        self.nationality = nationality
        # Convert to float directly
        self.scores = [float(score) for score in scores]
        # Sort and calculate final score in one pass
        sorted_scores = sorted(self.scores)
        self.finalScore = sum(sorted_scores[1:-1])
    
    def __str__(self):
        scores_str = " ".join(map(str, self.scores))
        return f"Athlete name: {self.name}\tsurname:{self.surname}\tnationality:{self.nationality}\tscores:{scores_str}\tfinal score:{self.finalScore}"

athletes = []

try:
    with open(filename, "r") as file:
        for line in file:
            parts = line.strip().split(" ")
            name = parts[0]
            surname = parts[1]
            nationality = parts[2]
            scores = parts[3:]
            athletes.append(Athlete(name, surname, nationality, scores))
except IOError:
    print("File not found error.")
except Exception as e:
    print(f"An error occurred: {e}")

# Sort and print top 3 athletes
athletes.sort(key=lambda x: x.finalScore, reverse=True)
for pos, a in enumerate(athletes[:3], start=1):
    print(f"{pos}: {a.name} {a.surname} - Score: {a.finalScore}")

# Calculate points by country more efficiently
country_scores = {}
for athlete in athletes:
    country_scores[athlete.nationality] = country_scores.get(athlete.nationality, 0) + athlete.finalScore

pointsByCountry = sorted(country_scores.items(), key=lambda x: x[1], reverse=True)
bestCountry, bestScore = pointsByCountry[0]
print(f"{bestCountry} - Total score: {bestScore}")


1: Donald Duck - Score: 28.7
2: Daffy Duck - Score: 27.3
3: Mickey Mouse - Score: 27.2
ITA - Total score: 55.9


#### Exercise 2 (adapted from Computer Sciences exam 23/06/2017)

You are asked to implement a program for managing the database of a city public transportation center.
The information is stored in a ﬁle whose name is passed as command line argument. Each line in the
ﬁle contains: the ID number of the public transport bus, the number of the route the bus serves, the
geometric coordinates in meters (abscissa and ordinate, i.e., x-axis and y-axis) and the time in seconds
(all the time values belong to the same day) when the bus is checked. For example, the ﬁle can contain:

In [76]:
import numpy as np

filename = "transport.txt"
data = np.empty((0,5), dtype= np.int32)

try:
    with open(filename, "r") as file:
        for line in file:
            data = np.vstack((data, 
                       np.array(line.strip().split(" "), dtype=np.int32)))
except IOError:
    print("File not found error.")
except Exception as e:
    print(f"An error occurred: {e}")

def total_distance_bus(data, busid):
    busData: np.array = data[data[:,0]==busid, 2:4]
    distance = 0.0
    for i in range(0, busData.shape[0]-1):
        distance += np.sqrt((busData[i,0] - busData[i+1,0])**2 + (busData[i,1] - busData[i+1,1])**2)
    return distance

def total_time_bus(data, busid):
    busData: np.array = data[data[:,0]==busid, 4]
    return max(busData)-min(busData)

def average_speed(data, lineid):
    lineData: np.array = data[data[:,1]==lineid,:]
    buses = np.unique(lineData[:,0])
    distance = 0.0
    time = 0.0
    for busid in buses:
        distance+=total_distance_bus(data, busid)
        time+=total_time_bus(data, busid)
    return distance/time

print(f"{1976} - Total Distance: {total_distance_bus(data, 1976)}")
print(f"{4} - Average Speed: {average_speed(data, 4)}")

1976 - Total Distance: 2195.0
4 - Average Speed: 1.6884615384615385


# Laboratory 1 - part II (numpy)

In [77]:
import numpy as np

#### Exercise 7
a. Write a function that, given two integers m and n, returns a m × n numpy 2D array with dtype numpy.float64 whose element in position i, j has value i ∗ j.
Example of return value for m = 3, n = 4:
array([[0., 0., 0., 0.],
    [0., 1., 2., 3.],
    [0., 2., 4., 6.]])

In [78]:
def ex7a(m,n):
    vr = np.reshape(np.arange(n), (1, n))
    vc = np.reshape(np.arange(m), (m, 1))
    return vc@vr

matrix = ex7a(5,6)
matrix +=1


b. Write a function that, given a 2D numpy array, computes a normalized version of the array, where the
normalization consists in scaling all the array columns so that the sum of the elements of each column is
one (assume that no column has a sum of elements that is zero, i.e., there’s no need to check for division
by zero). The function should not modify the input array, but should return a new array.

In [79]:

def ex7b(matrix):
    return matrix/np.reshape(np.sum(matrix, 0), (1, matrix.shape[1]))

print(ex7b(matrix))

[[0.2        0.06666667 0.04       0.02857143 0.02222222 0.01818182]
 [0.2        0.13333333 0.12       0.11428571 0.11111111 0.10909091]
 [0.2        0.2        0.2        0.2        0.2        0.2       ]
 [0.2        0.26666667 0.28       0.28571429 0.28888889 0.29090909]
 [0.2        0.33333333 0.36       0.37142857 0.37777778 0.38181818]]


c. Repeat the excercise b. but normalize the matrix rows so that the elements of each row sum up to one

In [80]:
def ex7c(matrix):
    return matrix/np.reshape(np.sum(matrix, 1), (matrix.shape[0],1))

print(ex7c(matrix))
matrix-=2 

[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.04761905 0.0952381  0.14285714 0.19047619 0.23809524 0.28571429]
 [0.02777778 0.08333333 0.13888889 0.19444444 0.25       0.30555556]
 [0.01960784 0.07843137 0.1372549  0.19607843 0.25490196 0.31372549]
 [0.01515152 0.07575758 0.13636364 0.1969697  0.25757576 0.31818182]]


d. Write a function that, given a numpy array of any shape, returns an array with the same shape but
all negative elements set to 0. The function should not modify the original array.

In [81]:
print(matrix)
def ex7d(matrix):
    nm = np.copy(matrix)
    nm[nm <0 ]=0
    return nm

nm=ex7d(matrix)
print(nm)

[[-1 -1 -1 -1 -1 -1]
 [-1  0  1  2  3  4]
 [-1  1  3  5  7  9]
 [-1  2  5  8 11 14]
 [-1  3  7 11 15 19]]
[[ 0  0  0  0  0  0]
 [ 0  0  1  2  3  4]
 [ 0  1  3  5  7  9]
 [ 0  2  5  8 11 14]
 [ 0  3  7 11 15 19]]


e. Write a function that computes the sum of the elements of a product of two matrices

In [87]:
def ex7e(m1, m2):
    return np.sum(m1*m2)

print(ex7e(matrix, nm))

1370


#### Exercise 8
Solve the ﬁrst part (ﬁnal ranking question) of Excercise 1 using numpy arrays to represent the scores
(ignore the best country question). Assume the number of competitors is given in the ﬁrst row of the
scores ﬁle.
Suggestion: You can store the competitors’ names in a list and the scores in a matrix, i.e. a 2D numpy
array, and use array operations to compute the actual score of each competitor and to ﬁnd the best
three competitors. For the last point, check the documentation on numpy.sort and numpy.argsort for
a simple way to extract the best three competitors.

In [171]:
from typing import List
import numpy as np

filename = "score.txt"

athletes_info: List[List[str]] = []
athletes_scores = np.zeros((1,5)) # Partire da None e poi creare l'array al primo inserimento

try:
    with open(filename, "r") as file:
        for line in file: 
            if line.strip()!="":
                parts = line.strip().split(" ", maxsplit=3)
                athletes_info.append(parts[:3])
                new_scores = [float(x) for x in parts[3].split(" ")]
                athletes_scores = np.vstack((athletes_scores, new_scores))
except IOError:
    print("File not found error.")
except Exception as e:
    print(f"An error occurred: {e}")

# Sort and print top 3 athletes
best_scores = np.sum(athletes_scores[:,1:4], 1, keepdims=True)
best_scores = best_scores[best_scores>0]
sorted_index = np.argsort(best_scores, stable=True)[::-1]

print(best_scores)
print(sorted_index)

print("Final ranking:")
for pos, i  in enumerate(sorted_index[:3], 1):
    print(f"{pos}. {athletes_info[i][0]} {athletes_info[i][1]} - Score: {best_scores[i]}")

[28.3 27.2 25.8 27.9 26.4]
[0 3 1 4 2]
Final ranking:
1. Donald Duck - Score: 28.3
2. Daffy Duck - Score: 27.9
3. Mickey Mouse - Score: 27.2
